In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🎭 Enhanced DistilBERT Sentiment Analysis - Demo Notebook\n",
    "\n",
    "This notebook demonstrates how to use the trained Enhanced DistilBERT model for sentiment analysis.\n",
    "\n",
    "**Model Performance:**\n",
    "- Accuracy: 95.17%\n",
    "- Dataset: SST-2 (67,349 samples)\n",
    "- Architecture: DistilBERT + LoRA Adapters\n",
    "- Training: Knowledge Distillation from BERT-base\n",
    "\n",
    "**Author:** Your Name  \n",
    "**Date:** 2024  \n",
    "**GitHub:** [Your Repository](https://github.com/yourusername/sentiment-analysis-distilbert)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📦 1. Setup and Installation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages (run only once)\n",
    "!pip install torch transformers pandas matplotlib seaborn plotly -q"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.nn.functional as F\n",
    "from transformers import DistilBertModel, DistilBertTokenizer\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from typing import Dict, List\n",
    "import warnings\n",
    "\n",
    "warnings.filterwarnings('ignore')\n",
    "sns.set_style('whitegrid')\n",
    "\n",
    "print(\"✅ All packages imported successfully!\")\n",
    "print(f\"PyTorch version: {torch.__version__}\")\n",
    "print(f\"CUDA available: {torch.cuda.is_available()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🏗️ 2. Model Architecture\n",
    "\n",
    "The model uses:\n",
    "- **LoRA (Low-Rank Adaptation)** for parameter-efficient fine-tuning\n",
    "- **Task-specific adapters** for enhanced performance\n",
    "- **Enhanced classifier head** with multiple layers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class LoRALayer(nn.Module):\n",
    "    \"\"\"Low-Rank Adaptation layer\"\"\"\n",
    "    def __init__(self, in_features: int, out_features: int, rank: int = 8, alpha: float = 16):\n",
    "        super().__init__()\n",
    "        self.rank = rank\n",
    "        self.scaling = alpha / rank\n",
    "        \n",
    "        self.lora_A = nn.Parameter(torch.zeros(in_features, rank))\n",
    "        self.lora_B = nn.Parameter(torch.zeros(rank, out_features))\n",
    "        \n",
    "        nn.init.kaiming_uniform_(self.lora_A, a=np.sqrt(5))\n",
    "        nn.init.zeros_(self.lora_B)\n",
    "    \n",
    "    def forward(self, x):\n",
    "        return (x @ self.lora_A @ self.lora_B) * self.scaling\n",
    "\n",
    "\n",
    "class EnhancedDistilBERT(nn.Module):\n",
    "    \"\"\"Enhanced DistilBERT with LoRA adapters\"\"\"\n",
    "    def __init__(self, model_name: str = \"distilbert-base-uncased\", num_labels: int = 2,\n",
    "                 adapter_size: int = 64, lora_rank: int = 8, dropout: float = 0.1):\n",
    "        super().__init__()\n",
    "        \n",
    "        self.distilbert = DistilBertModel.from_pretrained(model_name)\n",
    "        config = self.distilbert.config\n",
    "        hidden_size = config.hidden_size\n",
    "        num_layers = config.n_layers\n",
    "        \n",
    "        # Freeze base model\n",
    "        for param in self.distilbert.parameters():\n",
    "            param.requires_grad = False\n",
    "        \n",
    "        for name, param in self.distilbert.named_parameters():\n",
    "            if 'LayerNorm' in name:\n",
    "                param.requires_grad = True\n",
    "        \n",
    "        # LoRA layers\n",
    "        self.lora_layers = nn.ModuleList([\n",
    "            LoRALayer(hidden_size, hidden_size, rank=lora_rank)\n",
    "            for _ in range(num_layers)\n",
    "        ])\n",
    "        \n",
    "        # Adapters\n",
    "        self.adapters = nn.ModuleList([\n",
    "            nn.Sequential(\n",
    "                nn.Linear(hidden_size, adapter_size),\n",
    "                nn.GELU(),\n",
    "                nn.Dropout(dropout),\n",
    "                nn.Linear(adapter_size, hidden_size),\n",
    "                nn.LayerNorm(hidden_size)\n",
    "            ) for _ in range(num_layers)\n",
    "        ])\n",
    "        \n",
    "        # Classifier\n",
    "        self.classifier = nn.Sequential(\n",
    "            nn.Linear(hidden_size, hidden_size),\n",
    "            nn.LayerNorm(hidden_size),\n",
    "            nn.GELU(),\n",
    "            nn.Dropout(dropout * 2),\n",
    "            nn.Linear(hidden_size, hidden_size // 2),\n",
    "            nn.GELU(),\n",
    "            nn.Dropout(dropout),\n",
    "            nn.Linear(hidden_size // 2, num_labels)\n",
    "        )\n",
    "        \n",
    "        for module in self.classifier:\n",
    "            if isinstance(module, nn.Linear):\n",
    "                nn.init.xavier_uniform_(module.weight)\n",
    "                nn.init.zeros_(module.bias)\n",
    "    \n",
    "    def forward(self, input_ids, attention_mask):\n",
    "        outputs = self.distilbert(\n",
    "            input_ids=input_ids,\n",
    "            attention_mask=attention_mask,\n",
    "            output_hidden_states=True\n",
    "        )\n",
    "        \n",
    "        hidden_states = outputs.hidden_states\n",
    "        x = hidden_states[-1]\n",
    "        \n",
    "        for i, (lora, adapter) in enumerate(zip(self.lora_layers, self.adapters)):\n",
    "            layer_hidden = hidden_states[min(i + 1, len(hidden_states) - 1)]\n",
    "            x_lora = lora(layer_hidden)\n",
    "            x_adapter = adapter(layer_hidden)\n",
    "            x = layer_hidden + x_lora + x_adapter\n",
    "        \n",
    "        pooled = x[:, 0]\n",
    "        logits = self.classifier(pooled)\n",
    "        \n",
    "        return logits\n",
    "\n",
    "print(\"✅ Model architecture defined!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📥 3. Load Trained Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configuration\n",
    "MODEL_PATH = 'models/best_model_sst2.pt'\n",
    "DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "\n",
    "print(f\"Loading model from: {MODEL_PATH}\")\n",
    "print(f\"Using device: {DEVICE}\")\n",
    "\n",
    "# Load checkpoint\n",
    "checkpoint = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)\n",
    "\n",
    "# Initialize model\n",
    "model = EnhancedDistilBERT(num_labels=2)\n",
    "model.load_state_dict(checkpoint['model_state_dict'])\n",
    "model.to(DEVICE)\n",
    "model.eval()\n",
    "\n",
    "# Load tokenizer\n",
    "tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')\n",
    "\n",
    "# Model metadata\n",
    "accuracy = checkpoint.get('accuracy', 0.0)\n",
    "epoch = checkpoint.get('epoch', 0)\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"✅ MODEL LOADED SUCCESSFULLY!\")\n",
    "print(\"=\"*50)\n",
    "print(f\"📊 Validation Accuracy: {accuracy*100:.2f}%\")\n",
    "print(f\"📚 Training Epoch: {epoch}\")\n",
    "print(f\"🔧 Device: {DEVICE}\")\n",
    "\n",
    "# Count parameters\n",
    "total_params = sum(p.numel() for p in model.parameters())\n",
    "trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)\n",
    "print(f\"\\n📈 Total Parameters: {total_params:,}\")\n",
    "print(f\"🎯 Trainable Parameters: {trainable_params:,}\")\n",
    "print(f\"⚡ Parameter Efficiency: {100*trainable_params/total_params:.1f}%\")\n",
    "print(\"=\"*50)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔮 4. Prediction Function"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def predict_sentiment(text: str, verbose: bool = True) -> Dict:\n",
    "    \"\"\"\n",
    "    Predict sentiment for a given text\n",
    "    \n",
    "    Args:\n",
    "        text: Input text to analyze\n",
    "        verbose: Whether to print detailed output\n",
    "    \n",
    "    Returns:\n",
    "        Dictionary containing prediction results\n",
    "    \"\"\"\n",
    "    # Tokenize\n",
    "    inputs = tokenizer(\n",
    "        text,\n",
    "        return_tensors='pt',\n",
    "        padding=True,\n",
    "        truncation=True,\n",
    "        max_length=128\n",
    "    )\n",
    "    \n",
    "    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}\n",
    "    \n",
    "    # Predict\n",
    "    with torch.no_grad():\n",
    "        logits = model(inputs['input_ids'], inputs['attention_mask'])\n",
    "        probs = F.softmax(logits, dim=-1)\n",
    "        pred = probs.argmax(dim=-1)\n",
    "    \n",
    "    result = {\n",
    "        'text': text,\n",
    "        'sentiment': 'positive' if pred.item() == 1 else 'negative',\n",
    "        'label': int(pred.item()),\n",
    "        'confidence': float(probs[0][pred.item()]),\n",
    "        'probabilities': {\n",
    "            'negative': float(probs[0][0]),\n",
    "            'positive': float(probs[0][1])\n",
    "        }\n",
    "    }\n",
    "    \n",
    "    if verbose:\n",
    "        print(\"\\n\" + \"=\"*60)\n",
    "        print(\"📝 TEXT:\", text)\n",
    "        print(\"-\" * 60)\n",
    "        emoji = \"😊\" if result['sentiment'] == 'positive' else \"😞\"\n",
    "        print(f\"{emoji} SENTIMENT: {result['sentiment'].upper()}\")\n",
    "        print(f\"📊 Confidence: {result['confidence']*100:.2f}%\")\n",
    "        print(f\"📈 Probabilities:\")\n",
    "        print(f\"   • Negative: {result['probabilities']['negative']*100:.2f}%\")\n",
    "        print(f\"   • Positive: {result['probabilities']['positive']*100:.2f}%\")\n",
    "        print(\"=\"*60)\n",
    "    \n",
    "    return result\n",
    "\n",
    "\n",
    "def predict_batch(texts: List[str]) -> pd.DataFrame:\n",
    "    \"\"\"\n",
    "    Predict sentiment for multiple texts\n",
    "    \n",
    "    Args:\n",
    "        texts: List of texts to analyze\n",
    "    \n",
    "    Returns:\n",
    "        DataFrame with results\n",
    "    \"\"\"\n",
    "    results = []\n",
    "    \n",
    "    for text in texts:\n",
    "        result = predict_sentiment(text, verbose=False)\n",
    "        results.append(result)\n",
    "    \n",
    "    df = pd.DataFrame(results)\n",
    "    return df\n",
    "\n",
    "print(\"✅ Prediction functions ready!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎯 5. Example Predictions\n",
    "\n",
    "Let's test the model on various examples!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example 1: Positive sentiment\n",
    "result1 = predict_sentiment(\"This movie was absolutely fantastic! I loved every moment of it.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example 2: Negative sentiment\n",
    "result2 = predict_sentiment(\"Terrible film. Complete waste of time and money.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example 3: Neutral/Mixed sentiment\n",
    "result3 = predict_sentiment(\"The acting was good but the plot was confusing.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example 4: Your own text!\n",
    "custom_text = \"I'm so happy with this purchase! Highly recommend it.\"\n",
    "result4 = predict_sentiment(custom_text)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 6. Batch Analysis with Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Sample movie reviews\n",
    "sample_texts = [\n",
    "    \"This movie was absolutely fantastic!\",\n",
    "    \"Worst film I've ever seen.\",\n",
    "    \"Pretty good, would watch again.\",\n",
    "    \"Boring and predictable.\",\n",
    "    \"Amazing performances by the cast!\",\n",
    "    \"Not worth the ticket price.\",\n",
    "    \"One of the best films this year!\",\n",
    "    \"Disappointing ending.\",\n",
    "    \"Exceeded my expectations!\",\n",
    "    \"Could have been better.\"\n",
    "]\n",
    "\n",
    "# Get predictions\n",
    "results_df = predict_batch(sample_texts)\n",
    "\n",
    "# Display results\n",
    "print(\"\\n📋 BATCH PREDICTION RESULTS:\")\n",
    "print(\"=\"*80)\n",
    "display(results_df[['text', 'sentiment', 'confidence']].style.background_gradient(\n",
    "    subset=['confidence'], cmap='RdYlGn'\n",
    "))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize sentiment distribution\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "# Pie chart\n",
    "sentiment_counts = results_df['sentiment'].value_counts()\n",
    "colors = ['#38ef7d', '#eb3349']\n",
    "axes[0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',\n",
    "            colors=colors, startangle=90, textprops={'fontsize': 12, 'weight': 'bold'})\n",
    "axes[0].set_title('Sentiment Distribution', fontsize=14, weight='bold')\n",
    "\n",
    "# Confidence distribution\n",
    "axes[1].hist(results_df['confidence'], bins=10, color='#667eea', edgecolor='white', alpha=0.8)\n",
    "axes[1].set_xlabel('Confidence', fontsize=12)\n",
    "axes[1].set_ylabel('Frequency', fontsize=12)\n",
    "axes[1].set_title('Confidence Distribution', fontsize=14, weight='bold')\n",
    "axes[1].grid(axis='y', alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f\"\\n📊 Summary Statistics:\")\n",
    "print(f\"   Positive: {sentiment_counts.get('positive', 0)} ({sentiment_counts.get('positive', 0)/len(results_df)*100:.1f}%)\")\n",
    "print(f\"   Negative: {sentiment_counts.get('negative', 0)} ({sentiment_counts.get('negative', 0)/len(results_df)*100:.1f}%)\")\n",
    "print(f\"   Average Confidence: {results_df['confidence'].mean()*100:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎨 7. Interactive Probability Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "\n",
    "# Create subplots\n",
    "fig = make_subplots(\n",
    "    rows=1, cols=2,\n",
    "    subplot_titles=('Sentiment Probabilities', 'Confidence Scores'),\n",
    "    specs=[[{\"type\": \"bar\"}, {\"type\": \"scatter\"}]]\n",
    ")\n",
    "\n",
    "# Add probability bars\n",
    "for i, row in results_df.iterrows():\n",
    "    fig.add_trace(\n",
    "        go.Bar(\n",
    "            name=f\"Text {i+1}\",\n",
    "            x=['Negative', 'Positive'],\n",
    "            y=[row['probabilities']['negative'], row['probabilities']['positive']],\n",
    "            showlegend=False\n",
    "        ),\n",
    "        row=1, col=1\n",
    "    )\n",
    "\n",
    "# Add confidence scatter\n",
    "colors = ['#38ef7d' if s == 'positive' else '#eb3349' for s in results_df['sentiment']]\n",
    "fig.add_trace(\n",
    "    go.Scatter(\n",
    "        x=list(range(1, len(results_df)+1)),\n",
    "        y=results_df['confidence'],\n",
    "        mode='markers+lines',\n",
    "        marker=dict(size=12, color=colors, line=dict(width=2, color='white')),\n",
    "        line=dict(color='gray', width=1, dash='dot'),\n",
    "        showlegend=False\n",
    "    ),\n",
    "    row=1, col=2\n",
    ")\n",
    "\n",
    "fig.update_xaxes(title_text=\"Sentiment\", row=1, col=1)\n",
    "fig.update_xaxes(title_text=\"Text Number\", row=1, col=2)\n",
    "fig.update_yaxes(title_text=\"Probability\", row=1, col=1)\n",
    "fig.update_yaxes(title_text=\"Confidence\", row=1, col=2)\n",
    "\n",
    "fig.update_layout(height=400, showlegend=False, title_text=\"Prediction Analysis\")\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧪 8. Model Analysis & Insights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze model confidence on different sentiment strengths\n",
    "test_cases = {\n",
    "    'Strong Positive': [\n",
    "        \"Absolutely amazing! Best ever!\",\n",
    "        \"Incredible masterpiece!\",\n",
    "        \"Perfect in every way!\"\n",
    "    ],\n",
    "    'Mild Positive': [\n",
    "        \"Pretty good overall.\",\n",
    "        \"I liked it.\",\n",
    "        \"Worth watching.\"\n",
    "    ],\n",
    "    'Neutral': [\n",
    "        \"It was okay.\",\n",
    "        \"Not bad, not great.\",\n",
    "        \"Average experience.\"\n",
    "    ],\n",
    "    'Mild Negative': [\n",
    "        \"Could be better.\",\n",
    "        \"Not my favorite.\",\n",
    "        \"Somewhat disappointing.\"\n",
    "    ],\n",
    "    'Strong Negative': [\n",
    "        \"Absolutely terrible!\",\n",
    "        \"Worst ever!\",\n",
    "        \"Complete disaster!\"\n",
    "    ]\n",
    "}\n",
    "\n",
    "analysis_results = []\n",
    "\n",
    "for category, texts in test_cases.items():\n",
    "    for text in texts:\n",
    "        result = predict_sentiment(text, verbose=False)\n",
    "        analysis_results.append({\n",
    "            'Category': category,\n",
    "            'Text': text,\n",
    "            'Predicted': result['sentiment'],\n",
    "            'Confidence': result['confidence']\n",
    "        })\n",
    "\n",
    "analysis_df = pd.DataFrame(analysis_results)\n",
    "\n",
    "# Group by category\n",
    "category_stats = analysis_df.groupby('Category').agg({\n",
    "    'Confidence': ['mean', 'min', 'max']\n",
    "}).round(3)\n",
    "\n",
    "print(\"\\n📊 MODEL CONFIDENCE BY SENTIMENT STRENGTH:\")\n",
    "print(\"=\"*60)\n",
    "display(category_stats)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize confidence by category\n",
    "plt.figure(figsize=(12, 6))\n",
    "\n",
    "categories = list(test_cases.keys())\n",
    "avg_confidences = [analysis_df[analysis_df['Category'] == cat]['Confidence'].mean() \n",
    "                   for cat in categories]\n",
    "\n",
    "colors_map = plt.cm.RdYlGn(np.linspace(0, 1, len(categories)))\n",
    "bars = plt.bar(categories, avg_confidences, color=colors_map, edgecolor='white', linewidth=2)\n",
    "\n",
    "plt.xlabel('Sentiment Strength', fontsize=12, weight='bold')\n",
    "plt.ylabel('Average Confidence', fontsize=12, weight='bold')\n",
    "plt.title('Model Confidence Across Different Sentiment Strengths', fontsize=14, weight='bold')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.ylim(0, 1)\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "\n",
    "# Add value labels on bars\n",
    "for bar in bars:\n",
    "    height = bar.get_height()\n",
    "    plt.text(bar.get_x() + bar.get_width()/2., height,\n",
    "             f'{height:.2%}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💾 9. Export Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save batch results to CSV\n",
    "output_file = 'sentiment_analysis_results.csv'\n",
    "results_df.to_csv(output_file, index=False)\n",
    "print(f\"✅ Results saved to: {output_file}\")\n",
    "\n",
    "# Save analysis results\n",
    "analysis_output = 'sentiment_strength_analysis.csv'\n",
    "analysis_df.to_csv(analysis_output, index=False)\n",
    "print(f\"✅ Analysis saved to: {analysis_output}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎓 10. Try Your Own Text!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Interactive prediction - modify the text below\n",
    "YOUR_TEXT = \"Enter your text here to analyze sentiment!\"\n",
    "\n",
    "your_result = predict_sentiment(YOUR_TEXT)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📚 11. Model Information & Citation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\"\"\n",
    "╔═══════════════════════════════════════════════════════════════╗\n",
    "║        Enhanced DistilBERT Sentiment Analysis Model          ║\n",
    "╠═══════════════════════════════════════════════════════════════╣\n",
    "║                                                               ║\n",
    "║  Model Architecture: DistilBERT + LoRA Adapters              ║\n",
    "║  Base Model: distilbert-base-uncased                         ║\n",
    "║  Enhancement: Knowledge Distillation from BERT-base          ║\n",
    "║  Training Dataset: SST-2 (67,349 samples)                    ║\n",
    "║  Validation Accuracy: 95.17